In [1]:
import pandas as pd

df = pd.read_excel("data/input.xlsx")

In [2]:
df.head()

,timestamp,username,Album 1,Album 2,Album 3,Album 4,Album 5,Album 6,Album 7,Album 8,...,Album 21,Album 22,Album 23,Album 24,Album 25,Album 26,Album 27,Album 28,Album 29,Album 30
0,2021-12-04 00:32:21.677,Moland Fengkov,Mastodon – Hushed and Grim,Grey Aura – Zwart Vierkant,Papangu – Holoceno,Godspeed You Black Emperor – G_d’s Pee AT STAT...,Big|Brave – Vital,Cult of Dom Keller – They Carried The Dead In ...,Dvne – Etemen Aenka,Mythic Sunship – Wildfire,...,Cicada the Burrower – Corpseflower,Goat – Headsoup,Brii – Sem proposito,Mogwai – As the love continues,God is an Astronaut – Ghost tapes #10,Giöbia + The Cosmic Dead – The intergalactic c...,Primeval Well – Talkin’ in tongues with mounta...,Epiphanic Truth – Dark Triad : Bitter psalms t...,Jess and the Ancient Ones – Vertigo,Kauan – Ice fleet
1,2021-12-04 01:01:24.021,Léo Gerard,Amenra - De Doorne,Portal - Avow,Antediluvian - The Divine Punishment,MONO - The Pilgrimage of the Soul,Maybeshewill - No Feeling is Final,Portal - Hagbulbia,Cannibal Corpse - Violence Inimagined,Blockheads - Trip to the Void,...,Årabrot - Norwegian Gothic,Jours Pâles - Éclosion,Glasgow Coma Scale - Sirens,Inferi - Vile Genesis,Mare Cognitum - Solar Paroxysm,Mors Principium Est - Seven,Perturbator - Lustful Sacraments,SeeYouSpaceCowboy - Romance of Affliction,Tacoma Narrows Bridge Disaster - The World Inside,Violent Magic Orchestra - Principle of Light S...
2,2021-12-04 11:20:56.871,francois,SOEN - Imperial,HELLOWEEN - Helloween,WITHERFALL - The curse of Autumn,HANGING GARDEN - Skeleton Lake,MERCURY CIRCLE - Killing Moons,SILVER LAKE - s.l by Esa Holopanen,SWALLOW THE SUN - Moonflowers,IOTUNN - Access all worlds,...,SUBTERRANEAN MASQUERADE - Mountain fever,AEPHANEMER - A Dream Of Wilderness,BE'LAKOR - Coherence,A Pale Horse Named Death - Infernum In Terra,Insomnium - Argent Moon,NaN,NaN,NaN,NaN,NaN
3,2021-12-04 18:58:29.855,Caro Izinoff,Leprous - Aphelion,Idles - Crawler,Bruit - The machine is Burning and everyone kn...,Sunnata - Birning in heaven Melting on earth,BTBAM - Colors 2,Emma Ruth Rundle - Engine of Hell,Chevelle - Niratias,Fat Soul - Electric light and magic,...,Mastodon - Hushed and grim,Mono - Pilgrimage of the soul,Mr Marcaille - No snare No headache,Dvne - Etemen Aenka,Hundred Headless Horsemen - Apokalepsia,Greenleaf - Echoes from a mass,King woman - Celestial blues,Jess and the ancient ones - Vertigo,Vokonis - Odyssey,Psychedelic porn Crumpets - Shyga! The sunligh...
4,2021-12-05 16:31:27.312,Cyril Glaume,6:33 - Feary Tales For Strange Lullabies: The ...,Archspire - Bleed The Future,Sumo Cyco - Initiation,Ophidian I - Desolate,Diablo Swing Orchestra - Swagger & Stroll Down...,Krav Boca - Barrikade,First Fragment - Gloire Éternelle,Twelve Foot Ninja - Vengeance,...,Obsolete - Animate//Isolate,HAH - Chimaera Monstrosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
dfraw = df.drop(labels=['timestamp'],axis=1)
dfc = dfraw.fillna(0)

entry_refs = df['username']

entry_refs.head()

0    Moland Fengkov 
1         Léo Gerard
2           francois
3       Caro Izinoff
4       Cyril Glaume
Name: username, dtype: object

In [76]:
global_albums_set = set()

def process_entry_row(row):

    row_filled = row.notna()
    row_strip = row[row_filled == True]
    length = row_strip.shape[0] - 1
    print(length)

    global global_albums_set
    position = 1
    username = row_strip[0]

    entry_list = []

    for entry in row_strip[1:length+1]:

        entry_slug = entry.lower().strip(" ")
        global_albums_set.add(entry_slug)
        score = 25*(length - position)/(length - 1) + 5

        # print("ENTRY: {}".format(entry_slug))
        # print("POSITION: {}".format(position))
        # print("SCORE: {}".format(score))

        entry_list += {
            'username': username,
            'entry': entry_slug,
            'position': position,
            'score': score,
            'album': None,
            'genre': None
        },

        position += 1

    entry_df = pd.DataFrame(entry_list)
    
    return entry_df



In [77]:
entries_df = dfraw.apply(process_entry_row, axis=1)

# print(global_albums_set)

30
30
25
30
22
30
20
5
5
30
30
4
30
4
4
20
15
30
29
17
29
10
30
30
30
11
20
3
30
30
18
30
30
16
30
30
19
20
3
6
15
11
12
4
10
30
13
26
30
30
5
30
22
20
12
28


In [78]:
entries_df.iloc[12]

,username,entry,position,score,album,genre
0,Dee Cooper,swallow the sun - moonflowers,1,30.000000,None,None
1,Dee Cooper,cynic - ascension code,2,29.137931,None,None
2,Dee Cooper,grey aura - zwart vierkant,3,28.275862,None,None
3,Dee Cooper,cult of luna - the raging river,4,27.413793,None,None
4,Dee Cooper,dzung - dzanca,5,26.551724,None,None
5,Dee Cooper,lake of tears - omnious,6,25.689655,None,None
6,Dee Cooper,rivers of nihil - the work,7,24.827586,None,None
7,Dee Cooper,"sunnata - burning in heaven, melting on earth",8,23.965517,None,None
8,Dee Cooper,dream unending - tide turns eternal,9,23.103448,None,None
9,Dee Cooper,akhenaten - the emerald tablets of thoth,10,22.241379,None,None


In [79]:
albums = list(global_albums_set)

albums_df = pd.Series(albums)
albums_df.head()

albums_df.to_excel('data/albums_list.xlsx')

In [80]:
entries_df.shape

(56,)

In [81]:
entries = entries_df.iloc[0]

for i in range(1,entries_df.shape[0]):
    entries = pd.concat([entries,entries_df.iloc[i]])


In [82]:
entries

,username,entry,position,score,album,genre
0,Moland Fengkov,mastodon – hushed and grim,1,30.000000,None,None
1,Moland Fengkov,grey aura – zwart vierkant,2,29.137931,None,None
2,Moland Fengkov,papangu – holoceno,3,28.275862,None,None
3,Moland Fengkov,godspeed you black emperor – g_d’s pee at stat...,4,27.413793,None,None
4,Moland Fengkov,big|brave – vital,5,26.551724,None,None
...,...,...,...,...,...,...
23,Alex KM,mare cognitum - solar paroxysm,24,8.703704,None,None
24,Alex KM,first fragment - gloire eternelle,25,7.777778,None,None
25,Alex KM,mystras - empires vanquished and dismantled,26,6.851852,None,None
26,Alex KM,spectral lore - eterophotos,27,5.925926,None,None


In [85]:
def build_album_stats(entries):

   aggfunc = {
      'position': ['mean','min','max'],
      'score': ['count','sum','mean','max','min']
   }

   album_stats = pd.pivot_table(entries, index = ['entry'], values = ['score','position'], aggfunc=aggfunc)

   album_ranking = album_stats.rank(method='dense',ascending=False)[('score','sum')]
   # album_genre_ranking = album_stats.groupby('genre').rank(method='dense',ascending=False)[('score','sum')]

   album_results = pd.merge(album_stats, album_ranking, left_index=True, right_index=True)
   # album_results = pd.merge(album_results, album_genre_ranking, left_index=True, right_index=True)

   album_results.reset_index(inplace=True)

   return album_results

In [86]:
result = build_album_stats(entries)

KeyError: 'genre'